In [1]:
import os
import nltk
import pandas as pd
import re
import string
import pickle
import random
import numpy as np

from unidecode import unidecode
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import FreqDist, classify, NaiveBayesClassifier
from nltk.sentiment import SentimentIntensityAnalyzer

In [43]:
# nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\X308891\AppData\Roaming\nltk_data...


True

In [2]:
df = pd.read_csv(os.path.join('..', 'storage', 'base_sentiment.1.0.2.csv'))
input = open(os.path.join('..', 'storage', 'mac_morpho.1.0.0.pkl'), 'rb')
tagger = pickle.load(input)
input.close()

In [3]:
stopwords = set(nltk.corpus.stopwords.words('portuguese'))
lemmatizer = WordNetLemmatizer()
token_whitespace = nltk.tokenize.WhitespaceTokenizer()
stemmer = nltk.RSLPStemmer()

In [4]:
test = dict(text=df['text'].values, sentiment=df['sentiment'].values)

In [5]:
max_length = 160
texts = {'text': [], 'sentiment': []}


def complete_text(text: str) -> str:
    text = text + ' '
    left = max_length - len(text)
    
    for _ in range(left):
        text = text + '#'
    
    return text

for i, word in enumerate(test['text']):
    for index in range(0, len(word), max_length):
        if len(word[index:]) < max_length:
            texts['text'].append(complete_text(word[index:]))
            texts['sentiment'].append(test.get('sentiment')[i])
        else: 
            texts['text'].append(word[index:(index + max_length)])
            texts['sentiment'].append(test.get('sentiment')[i])

df_texts = pd.DataFrame(texts)

In [6]:
df_texts.head()

,text,sentiment
0,"Mais uma vez, o Sr. Costner arrumou um filme p...",negative
1,"s, eu simplesmente não me importei com nenhum ...",negative
2,"epois esquecido até muito mais tarde, quando e...",negative
3,cher. O problema é que ele sai como um garoto ...,negative
4,u único obstáculo parece estar vencendo Costne...,negative


In [7]:
#step one
def step1(text):
    return unidecode(text.lower())

In [ ]:
# step2 = lambda token: re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
# step3 = lambda token: re.sub("(@[A-Za-z0-9_]+)","", token)

In [8]:
def step2(text):
    return re.sub(r'\w*\d\w*', '', text)  # remover caractres especiais

In [9]:
def step3(text):
    return re.sub(r'[%s]' % re.escape(string.punctuation), ' ', text)  # remover pontuações

In [10]:
def step4(text):
    return text.replace('  ', ' ')  # remover espaço duplos

In [11]:
def step5(text):
    words = []

    for sentence in tagger.tag(token_whitespace.tokenize(text)):
        word, tag = sentence
        if tag.startswith("N"):
            pos = 'n'
        elif tag.startswith('V'):
            pos = 'v'
        else:
            pos = 'a'
        lemm = lemmatizer.lemmatize(word, pos)
        words.append(lemm)
            
    return ' '.join(words)

In [12]:
def step6(text):
    return " ".join(text for text in text.split() if text not in stopwords and len(text) > 0)

In [13]:
def step7(text):
    return stemmer.stem(text)

In [42]:
df_texts['text_clean'] = df_texts['text'].map(step1).map(step2).map(step3).map(step4).map(step5).map(step6)

In [43]:
df_texts['text_clean']

0         vez sr costner arrumou filme tempo necessario ...
1         s simplesmente nao importei nenhum personagens...
2         epois esquecido ate tarde nao importava person...
3         cher problema sai garoto pensa melhor qualquer...
4         u unico obstaculo parece vencendo costner fina...
                                ...                        
420887    voce pode adivinhar resto adaptacao romance pa...
420888    s bagunca ainda agradavel fantasia colarinho a...
420889    apenas ferramenta enredo historia amor segue n...
420890    ncolores derivacoes reduzidas nivel interessan...
420891    am final feliz foto nao realmente satisfatoria...
Name: text_clean, Length: 420892, dtype: object

In [44]:
freq = FreqDist(token_whitespace.tokenize(' '.join([text for text in df_texts['text_clean']])))

In [45]:
freq.most_common(n=5)

[('filme', 150476),
 ('nao', 145599),
 ('voce', 56709),
 ('sao', 35007),
 ('filmes', 31634)]

In [ ]:
# train_df = df_texts.sample(frac=0.8, random_state=100)
# test_df = df_texts[~df_texts.index.isin(train_df.index)]

In [46]:
positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []

for w in df_texts.query('sentiment == "positive"')['text_clean'].values:
        positive_cleaned_tokens_list.append(token_whitespace.tokenize(w))

for w in df_texts.query('sentiment == "negative"')['text_clean'].values:
        negative_cleaned_tokens_list.append(token_whitespace.tokenize(w))

In [47]:
def document_features(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

In [48]:
positive_dataset = [(tweet_dict, "Positive") for tweet_dict in document_features(positive_cleaned_tokens_list)]
negative_dataset = [(tweet_dict, "Negative") for tweet_dict in document_features(negative_cleaned_tokens_list)]
dataset = positive_dataset + negative_dataset

random.shuffle(dataset)

In [49]:
train, validate, test = np.split(dataset, [int(len(dataset)*0.8), int(len(dataset)*0.2)])

In [50]:
# Accuracy is: 0.7879832736387558 => sem stemmer
# Accuracy is: 0.7873744483448861 => sem stemmer
# Accuracy is: 0.7872586230450769 => com stemmer
# Accuracy is: 0.7874219664166028 => com stemmer on position 4
# Accuracy is: 0.7869289664225426 => com stemmer on position 2
# Accuracy is: 0.7867062254613708 => com stemmer on position 0
classifier = NaiveBayesClassifier.train(train)

print("Accuracy is:", classify.accuracy(classifier, test))
print(classifier.show_most_informative_features(10))

Accuracy is: 0.7873744483448861
Most Informative Features
              desperdice = True           Negati : Positi =     51.7 : 1.0
                    boll = True           Negati : Positi =     43.5 : 1.0
              redentores = True           Negati : Positi =     33.9 : 1.0
                   fedor = True           Negati : Positi =     30.0 : 1.0
                     nan = True           Positi : Negati =     25.5 : 1.0
                   anand = True           Positi : Negati =     24.9 : 1.0
               crossfire = True           Positi : Negati =     24.9 : 1.0
                   secar = True           Negati : Positi =     23.9 : 1.0
                  gotham = True           Positi : Negati =     23.5 : 1.0
                   lindy = True           Positi : Negati =     22.9 : 1.0
None


In [39]:
tt = pd.Series(['A vida nem sempre é fácil, mas muitas vezes cabe a nós mudar nossa perspectiva e enxergá-la de forma mais positiva. Não importa o que você está passando agora, contanto que nunca perca a esperança nas coisas boas que estão por vir ']).map(step1).map(step2).map(step3).map(step4).map(step5)
tt[0]

'a vida nem sempre e facil mas muitas vezes cabe a nos mudar nossa perspectiva e enxerga la de forma mais positiva nao importa o que voce esta passando agora contanto que nunca perca a esperanca na coisas boas que estao por vir'

In [52]:
pd.Series(token_whitespace.tokenize(tt[0])).map(step7)

0            a
1          vid
2          nem
3        sempr
4            e
5        facil
6          mas
7         muit
8          vez
9          cab
10           a
11          no
12         mud
13        noss
14     perspec
15           e
16      enxerg
17          la
18          de
19        form
20        mais
21        posi
22         nao
23      import
24           o
25         que
26         voc
27         est
28        pass
29        agor
30     contant
31         que
32        nunc
33        perc
34           a
35    esperanc
36          na
37        cois
38         boa
39         que
40        esta
41         por
42         vir
dtype: object

In [64]:
negative_frase = [
    'Ontem eu não fui ao dentista.'
    'Meu pai nunca gostou daquela cantora.',
    'O vidro da janela nem está muito sujo.',
    'Eu jamais pedirei sua ajuda.',
    'Eu tampouco fui selecionada no processo de seleção da empresa.'
]

In [66]:
for text in negative_frase:
    predict = classifier.classify(dict([token, True] for token in [text]))
    # predict = sia.polarity_scores("Ontem eu não fui ao dentista.")

    print(f'predict: {predict} => {text}')

predict: Positive => Ontem eu não fui ao dentista.Meu pai nunca gostou daquela cantora.
predict: Positive => O vidro da janela nem está muito sujo.
predict: Positive => Eu jamais pedirei sua ajuda.
predict: Positive => Eu tampouco fui selecionada no processo de seleção da empresa.
